In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold

In [3]:
sample_submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
train_set = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_set = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [4]:
correlated_features = set()
correlation_matrix = train_set.corr()
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
len(correlated_features)

4

In [5]:
train_set.drop(labels=correlated_features, axis=1, inplace=True)
test_set.drop(labels=correlated_features, axis=1, inplace=True)

In [6]:
with open('/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt', 'r') as f:
    print(f.read())

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM

In [7]:
y_train = train_set['SalePrice'].copy()

In [8]:
train_set

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [9]:
train_set.drop(['SalePrice'], axis=1, inplace=True)

In [10]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [11]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])
num_attribs = list(train_set.select_dtypes(exclude=['object']))
cat_attribs = list(train_set.select_dtypes(include=['object']))
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_attribs)
])
housing_prepared = full_pipeline.fit_transform(train_set)

In [12]:
type(housing_prepared)

scipy.sparse.csr.csr_matrix

In [13]:
#Linear Regression
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, y_train)
some_data = train_set.iloc[:5]
some_labels = y_train.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
lin_reg.predict(some_data_prepared)

array([206474.5265018 , 205181.43192302, 206032.23047329, 166649.02098293,
       287472.43184683])

In [14]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

20583.468441241737

In [15]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, y_train)
tree_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(y_train, housing_predictions)
tree_rsme = np.sqrt(tree_mse)
tree_rsme

20583.468441241737

In [16]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, y_train, scoring='neg_mean_squared_error', cv=3)
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

array([35548.58786294, 40578.54963589, 47442.382296  ])

In [17]:
print("Mean: ", tree_rmse_scores.mean())
print("Standard deviation:", tree_rmse_scores.std())

Mean:  41189.83993161022
Standard deviation: 4874.822620580863


In [18]:
lin_scores = cross_val_score(lin_reg, housing_prepared, y_train, scoring='neg_mean_squared_error', cv=3)
lin_rmse_scores = np.sqrt(-lin_scores)
lin_rmse_scores

array([28091.6568944 , 33687.97428052, 40761.64964785])

In [19]:
print("Mean: ", lin_rmse_scores.mean())
print("Standard deviation:", lin_rmse_scores.std())

Mean:  34180.42694092387
Standard deviation: 5184.210728109532


In [20]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, y_train)
forest_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(y_train, housing_predictions)
forest_rsme = np.sqrt(forest_mse)
forest_rsme

20583.468441241737

In [21]:
forest_scores = cross_val_score(forest_reg, housing_prepared, y_train, scoring='neg_mean_squared_error', cv=3)
forest_rmse_scores = np.sqrt(-forest_scores)
forest_rmse_scores

array([26579.07139545, 31473.4475588 , 31930.08704489])

In [22]:
print("Mean: ", forest_rmse_scores.mean())
print("Standard deviation:", forest_rmse_scores.std())

Mean:  29994.20199971458
Standard deviation: 2422.0470251703414


In [23]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators':[3,10,30], 'max_features':[2,4,6,8]},
    {'bootstrap':[False], 'n_estimators':[3,10], 'max_features':[2,3,4]}
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring = 'neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [24]:
grid_search.best_estimator_

RandomForestRegressor(max_features=8, n_estimators=30)

In [25]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

46020.886054982606 {'max_features': 2, 'n_estimators': 3}
39416.041893127316 {'max_features': 2, 'n_estimators': 10}
39367.9486489362 {'max_features': 2, 'n_estimators': 30}
44926.59467323876 {'max_features': 4, 'n_estimators': 3}
40481.69063141038 {'max_features': 4, 'n_estimators': 10}
37742.69981772612 {'max_features': 4, 'n_estimators': 30}
44730.986459868145 {'max_features': 6, 'n_estimators': 3}
37738.9098305461 {'max_features': 6, 'n_estimators': 10}
36236.10270710689 {'max_features': 6, 'n_estimators': 30}
42742.658589605 {'max_features': 8, 'n_estimators': 3}
37228.55996211899 {'max_features': 8, 'n_estimators': 10}
34071.92613707995 {'max_features': 8, 'n_estimators': 30}
48122.03228169317 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
39558.13719658986 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
46544.79609375395 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
36569.64508626609 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10}

In [26]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([7.64810852e-03, 4.93747131e-03, 2.40794253e-02, 2.30377481e-02,
       3.60866339e-02, 6.36399040e-03, 2.66990832e-02, 1.72313850e-02,
       2.30160390e-02, 2.59546064e-02, 1.43114056e-03, 1.07324265e-02,
       4.24617716e-02, 2.58693544e-02, 2.91433070e-04, 5.72071973e-02,
       7.36038499e-03, 3.50424981e-04, 2.59580538e-02, 3.03980344e-03,
       8.10919842e-03, 7.12843049e-04, 3.00773930e-02, 1.44078779e-02,
       1.14040890e-02, 1.52332990e-02, 1.79949442e-03, 3.66025538e-04,
       1.90790061e-03, 2.29356270e-03, 4.55083717e-04, 6.96102541e-03,
       3.97106053e-03, 2.43904326e-04, 9.36095875e-04, 4.13090851e-05,
       1.91123267e-03, 4.74121056e-03, 9.31219689e-05, 4.40146986e-05,
       6.76163146e-04, 2.95185089e-04, 5.86174573e-04, 4.09057951e-03,
       6.76773068e-04, 3.49152914e-04, 2.28793373e-03, 6.44959702e-04,
       8.22059336e-04, 7.16859126e-04, 1.92993242e-03, 3.48753827e-06,
       1.18408006e-06, 1.26744799e-03, 3.03464561e-03, 2.91276429e-04,
      

In [27]:
cat_encoder = full_pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)
# maybe drop less useful features

[(0.05720719730686292, 'GrLivArea'),
 (0.04246177155967001, 'TotalBsmtSF'),
 (0.03608663386496283, 'OverallQual'),
 (0.0300773930052748, 'Fireplaces'),
 (0.026699083219056504, 'YearBuilt'),
 (0.025958053822974746, 'FullBath'),
 (0.025954606389930624, 'BsmtFinSF1'),
 (0.02586935437390636, '2ndFlrSF'),
 (0.024079425277484263, 'LotFrontage'),
 (0.023037748110662584, 'LotArea'),
 (0.023016038994599856, 'MasVnrArea'),
 (0.017231385026205102, 'YearRemodAdd'),
 (0.015233298995386878, 'OpenPorchSF'),
 (0.014407877855505383, 'GarageCars'),
 (0.011404088993545036, 'WoodDeckSF'),
 (0.01073242647989959, 'BsmtUnfSF'),
 (0.008109198415812553, 'BedroomAbvGr'),
 (0.007648108520665905, 'Id'),
 (0.007360384992726249, 'BsmtFullBath'),
 (0.006961025412434693, 'MoSold'),
 (0.006363990399996471, 'OverallCond'),
 (0.004937471313114847, 'MSSubClass'),
 (0.004741210561665671, 'RM'),
 (0.0039710605326254, 'YrSold'),
 (0.0030398034411270388, 'HalfBath'),
 (0.0022935626956226843, 'PoolArea'),
 (0.0019112326658729

In [28]:
final_model = grid_search.best_estimator_
X_test= test_set.copy()

In [29]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [30]:


X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)
final_predictions

array([131078.33333333, 161541.66666667, 193456.06666667, ...,
       153190.        , 118825.2       , 211983.33333333])

In [31]:
test_ID = test_set['Id']
sub = pd.DataFrame({'Id':test_ID, 'SalePrice':final_predictions})
sub

,Id,SalePrice
0,1461,131078.333333
1,1462,161541.666667
2,1463,193456.066667
3,1464,203051.666667
4,1465,180221.666667
...,...,...
1454,2915,95490.000000
1455,2916,103930.000000
1456,2917,153190.000000
1457,2918,118825.200000


In [32]:
sub.to_csv('submission.csv', index=False)